In [1]:
##masking images and saving it subfolderwise

import os
import json
import shutil #to create subfolders 
from PIL import Image
from PIL import ImageDraw

def crop_polygons(image_path, json_path, output_folder):
    # Load JSON file
    with open(json_path, 'r') as file:
        data = json.load(file)

    # Open the image
    image = Image.open(image_path)

    # Counter variable for contour index
    contour_index = 0  # to assign a unique index to each contour while saving them as separate cropped images.

    # Iterate over each shape in the JSON file
    for shape in data['shapes']:
        label = shape['label']
        points = shape['points']

        # Convert points to integers
        points = [(int(x), int(y)) for x, y in points]

        # Create a new image with white background
        cropped_image = Image.new('RGBA', image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(cropped_image)

        # Draw the polygon on the new image
        draw.polygon(points, fill=(255, 255, 255, 255))

        # Copy the original image within the polygon bounds
        mask = Image.new('L', image.size, 0) 
        # to create a new image (mask) with the same size as the input image, initialized with a black (zero) intensity value.
        #'L' stands for grayscale mode,the image will have only a single channel representing the intensity values ranging from 0 to 255.
        ImageDraw.Draw(mask).polygon(points, outline=1, fill=1)
        cropped_image.paste(image, (0, 0), mask=mask)

        # Create a subfolder for the image (if not already exists)
        image_filename = os.path.splitext(os.path.basename(image_path))[0]
        image_subfolder = os.path.join(output_folder, image_filename)
        os.makedirs(image_subfolder, exist_ok=True)

        # Save the cropped image within the contour folder
        output_filename = f"{label}_{contour_index}_cropped.png"
        output_path = os.path.join(image_subfolder, output_filename)
        cropped_image.save(output_path)

        contour_index += 1

    print(f'Saved cropped images for {image_path}')

# Input folder containing .bmp files and JSON files
input_folder = 'C:/Users/dhyan/Downloads/data/'

# Output folder for the cropped images
output_folder = 'C:/Users/dhyan/Downloads/data/output_mask/'

# Iterate over each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        image_path = os.path.join(input_folder, filename) #to concatenate the paths
        json_path = os.path.join(input_folder, filename.replace('.png', '.json'))

        # Create subfolders and crop the polygons
        crop_polygons(image_path, json_path, output_folder)

        #here you will get the outputs of each product of image in different folders.

Saved cropped images for C:/Users/dhyan/Downloads/data/oval.png
